### Hbase的数据结构

#### 1. Rowkey
唯一标识一行记录的主键, Hbase按照RowKey的字典顺序全局排序
#### 2. 稀疏矩阵
Hbase中表的数据按照稀疏方式存储. 因为不想传统数据库组织数据形式, Hbase的列组成很灵活, 行与行之间不遵循相同的列定义
#### 3. Region
Hbase不同于spark的hash code分区, 使用"key range"划分子表"Region". Region是Hbase负载均衡的基本单元, Region增大到一定大小后会自动分裂
#### 4. column family
Region是对Hbase表的横向切割, 则Column Family是对表的纵向切割. 每个column都要归属于一个Column Family, 这个归属关系是在写数据时指定的而不是建表时预先定义的
#### 5. kv形式存储
Hbase中每行的每一列数据都以key-value形式存储. key相同的任意数量的独立key-value形成逻辑上的一行数据

[一条数据的HBase之旅，简明HBase入门教程-开篇](https://mp.weixin.qq.com/s/CXsGcbbsKTMXotlwRFQ5xw)

### Hbase行键设计

往往组合多个字段在一起作为rowkey, RowKey中的第一个字段称之为"先导字段"

#### 1. 反转
如果先到字段本身会带来热点问题, 但该字段的尾部却具有良好的随机性; 此时可将先导字段做反转处理

#### 2. 加盐
在rowkey前面加上长度固定的随机Bytes

#### 3. 哈希
将rowkey哈希后生成固定长度的byte串. 由于hash有很好的离散度, 所以可保障数据被均匀分散到各个region. hash rowkey既适合于随机写入, 也适合于知道rowkey后的随机查询, 带在查询一个范围内的rowkey时会有性能问题, 因为原本连续的rowkeyhash后被离散到多个region


### 列的设计
hbase中的每一列都是key-value形式的. 定义列就是要定义出列簇和列标识符. 列簇和列标识符应该尽量短
<img src="img/liecu.PNG" height="50%" width="50%">
[一条数据的HBase之旅，简明HBase入门教程-Write全流程](https://mp.weixin.qq.com/s/cpsX0j7IVfi54CjVWpGoqg?) 


### 写的过程
首先我们需要知道 HBase 的集群是通过 Zookeeper 来进行机器之前的协调，也就是说 HBase Master 与 Region Server 之间的关系是依赖 Zookeeper 来维护。当一个 Client 需要访问 HBase 集群时，Client 需要先和 Zookeeper 来通信，然后才会找到对应的 Region Server。每一个 Region Server 管理着很多个 Region。对于 HBase 来说，Region 是 HBase 并行化的基本单元。因此，数据也都存储在 Region 中。这里我们需要特别注意，每一个 Region 都只存储一个 Column Family 的数据，并且是该 CF 中的一段（按 Row 的区间分成多个 Region）。Region 所能存储的数据大小是有上限的，当达到该上限时（Threshold），Region 会进行分裂，数据也会分裂到多个 Region 中，这样便可以提高数据的并行化，以及提高数据的容量。每个 Region 包含着多个 Store 对象。每个 Store 包含一个 MemStore，和一个或多个 HFile。MemStore 便是数据在内存中的实体，并且一般都是有序的。当数据向 Region 写入的时候，会先写入 MemStore。当 MemStore 中的数据需要向底层文件系统倾倒（Dump）时（例如 MemStore 中的数据体积到达 MemStore 配置的最大值），Store 便会创建 StoreFile，而 StoreFile 就是对 HFile 一层封装。所以 MemStore 中的数据会最终写入到 HFile 中，也就是磁盘 IO。由于 HBase 底层依靠 HDFS，因此 HFile 都存储在 HDFS 之中。这便是整个 HBase 工作的原理简述。

我们了解了 HBase 大致的工作原理，那么在 HBase 的工作过程中，如何保证数据的可靠性呢？带着这个问题，我们理解下 HLog 的作用。HBase 中的 HLog 机制是 WAL 的一种实现，而 WAL（一般翻译为预写日志）是事务机制中常见的一致性的实现方式。每个 Region Server 中都会有一个 HLog 的实例，Region Server 会将更新操作（如 Put，Delete）先记录到 WAL（也就是 HLog）中，然后将其写入到 Store 的 MemStore，最终 MemStore 会将数据写入到持久化的 HFile 中（MemStore 到达配置的内存阀值）。这样就保证了 HBase 的写的可靠性。如果没有 WAL，当 Region Server 宕掉的时候，MemStore 还没有写入到 HFile，或者 StoreFile 还没有保存，数据就会丢失。或许有的读者会担心 HFile 本身会不会丢失，这是由 HDFS 来保证的。在 HDFS 中的数据默认会有 3 份。因此这里并不考虑 HFile 本身的可靠性。

rowkey设计: hashcode(用户ID)^时间^sessionId

[一条数据的HBase之旅，简明HBase入门教程-Flush与Compaction](https://mp.weixin.qq.com/s/ctnCm3uLCotgRpozbXmVMg?)